# Stacking

In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE   # SMOTE采样
from collections import Counter
from sklearn.ensemble import AdaBoostClassifier
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')


col_list = pd.read_csv('./指标/300次训练.csv',encoding='gbk',header=None)
new_col = list(col_list[col_list[0]=='制造业'][1])
new_col.append('TICKER_SYMBOL')
new_col.append('FLAG')
os.getcwd()  # 'C:\\Users\\dell\\zhaoxuan'
os.listdir()  # 列出所有文件
os.chdir('./清理后的数据/')
filelist = os.listdir()

csv_list = [x for x in filelist if '数据.csv' in x]
print(csv_list)
print(csv_list[0])
data_rate = pd.read_csv(csv_list[4])
data_rate = data_rate[new_col]
data_X = data_rate[~data_rate['FLAG'].isnull()]
data_Y = data_rate[data_rate['FLAG'].isnull()]
data_X['FLAG'] = data_X['FLAG'].apply(lambda x :int(x))
X_1 = data_X[data_X['FLAG']==1]
X_0 = data_X[data_X['FLAG']==0].sample(1000)
X_0 = X_0.append(X_1)
X_tr, X_te, Y_tr, Y_te = train_test_split(X_0.iloc[:,:-2],X_0.iloc[:,-1],test_size=0.25)   # 拆分训练集、测试集,12,134

sos = SMOTE(random_state=0)  # SMOTE采样
X_sos, y_sos = sos.fit_resample(X_tr, Y_tr)

['交通运输、仓储和邮政业清洗完成的数据.csv', '住宿和餐饮业清洗完成的数据.csv', '信息传输、软件和信息技术服务业清洗完成的数据.csv', '农、林、牧、渔业清洗完成的数据.csv', '制造业清洗完成的数据.csv', '卫生和社会工作清洗完成的数据.csv', '居民服务、修理和其他服务业清洗完成的数据.csv', '建筑业清洗完成的数据.csv', '房地产业清洗完成的数据.csv', '批发和零售业清洗完成的数据.csv', '教育清洗完成的数据.csv', '文化、体育和娱乐业清洗完成的数据.csv', '水利、环境和公共设施管理业清洗完成的数据.csv', '电力、热力、燃气及水生产和供应业清洗完成的数据.csv', '科学研究和技术服务业清洗完成的数据.csv', '租赁和商务服务业清洗完成的数据.csv', '综合清洗完成的数据.csv', '采矿业清洗完成的数据.csv', '金融业清洗完成的数据.csv']
交通运输、仓储和邮政业清洗完成的数据.csv


In [2]:
#加载需要的模块
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error,  make_scorer
from xgboost.sklearn import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from lightgbm.sklearn import LGBMClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
import lightgbm as lgb
import xgboost as xgb
from sklearn.model_selection import GridSearchCV,cross_val_score
from sklearn import linear_model
from sklearn.metrics import roc_curve, auc ,roc_auc_score

In [3]:
#定义模型
def build_model_gbdt(x_train,y_train):
    estimator =GradientBoostingClassifier(random_state=2)
    param_grid = { 
            'learning_rate': [0.05,0.08,0.1,0.2],
            }
    gbdt = GridSearchCV(estimator, param_grid,cv=3)
    gbdt.fit(x_train,y_train)
    print(gbdt.best_params_)
    # print(gbdt.best_estimator_ )
    return gbdt

def build_model_xgb(x_train,y_train):
    model = xgb.XGBClassifier(n_estimators=120, learning_rate=0.08) #, objective ='reg:squarederror', gamma=0, subsample=0.8,\        colsample_bytree=0.9, max_depth=5

    model.fit(x_train, y_train)
    return model

def build_model_lgb(x_train,y_train):
    estimator = lgb.LGBMClassifier(num_leaves=63,n_estimators = 100)
    param_grid = {
        'learning_rate': [0.01, 0.05, 0.1],
    }
    gbm = GridSearchCV(estimator, param_grid)
    gbm.fit(x_train, y_train)
    return gbm


def build_model_RF(x_train,y_train):
    reg_model = RandomForestClassifier(max_depth=4, random_state=2,n_estimators=120)
    reg_model.fit(x_train,y_train)
    return reg_model

def build_model_LR(x_train,y_train):
    LR_model = linear_model.LogisticRegression()
    LR_model.fit(x_train,y_train)
    return LR_model

In [4]:
#交叉验证
#划分训练集和测试集
x_train,x_val,y_train,y_val = train_test_split(X_0.iloc[:,:-2],X_0.iloc[:,-1],test_size=0.25) 

sos = SMOTE(random_state=0)  # SMOTE采样
x_train, y_train = sos.fit_resample(x_train, y_train)

#训练模型
print('Predict GBDT...')
model_gbdt = build_model_gbdt(x_train,y_train)
val_gbdt = model_gbdt.predict(x_val)
subA_gbdt = model_gbdt.predict(data_Y.iloc[:,:-2])

print('predict XGB...')
model_xgb = build_model_xgb(x_train,y_train)
val_xgb = model_xgb.predict(x_val)
subA_xgb = model_xgb.predict(data_Y.iloc[:,:-2])
y_score_xgb = model_xgb.predict_proba(x_val)[:,1]
fpr_xgb,tpr_xgb,thresholds_gbdt = roc_curve(y_val, y_score_xgb, pos_label=1)
roc_auc_xgb = auc(fpr_xgb,tpr_xgb)  # 计算auc的值
print(roc_auc_xgb)

print('predict lgb...')
model_lgb = build_model_lgb(x_train,y_train)
val_lgb = model_lgb.predict(x_val)
subA_lgb = model_lgb.predict(data_Y.iloc[:,:-2])

print('Predict LR...')
model_LR = build_model_LR(x_train,y_train)
val_gbdt = model_LR.predict(x_val)
subA_gbdt = model_LR.predict(data_Y.iloc[:,:-2])

Predict GBDT...
{'learning_rate': 0.2}
predict XGB...
[13:04:22] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
0.8150698030216101
predict lgb...


In [5]:
#Starking
#第一层
train_lgb_pred = model_lgb.predict(x_train)
train_xgb_pred = model_xgb.predict(x_train)
train_gbdt_pred = model_gbdt.predict(x_train)
train_LR_pred = model_LR.predict(x_train)

Strak_X_train = pd.DataFrame()
Strak_X_train['Method_1'] = train_lgb_pred
Strak_X_train['Method_2'] = train_xgb_pred
Strak_X_train['Method_3'] = train_gbdt_pred
Strak_X_train['Method_4'] = train_LR_pred

Strak_X_val = pd.DataFrame()
Strak_X_val['Method_1'] = val_lgb
Strak_X_val['Method_2'] = val_xgb
Strak_X_val['Method_3'] = val_gbdt
Strak_X_val['Method_4'] = val_LR

Strak_X_test = pd.DataFrame()
Strak_X_test['Method_1'] = subA_lgb
Strak_X_test['Method_2'] = subA_xgb
Strak_X_test['Method_3'] = subA_gbdt
Strak_X_test['Method_4'] = subA_LR

In [6]:
#第二层
model_lr_Stacking = build_model_RF(Strak_X_train,y_train)

#训练集
train_pre_Stacking = model_lr_Stacking.predict(Strak_X_train)
# roc_auc_score(Strak_X_val, model_lr_Stacking.decision_function(Strak_X_train))
# y_score_1 = model_lr_Stacking.predict_proba(Strak_X_val)[:,1]
# fpr_1,tpr_1,thresholds_q = roc_curve(y_val, y_score_1, pos_label=1)
# roc_auc_1 = auc(fpr_1,tpr_1)  # 计算auc的值
# print('ROC-AUC:',roc_auc_1)


#验证集
val_pre_Stacking = model_lr_Stacking.predict(Strak_X_val)
# print('ROC-AUC:',mean_absolute_error(y_val,val_pre_Stacking))

#预测集
print('Predict Stacking-RF...')
subA_Stacking = model_lr_Stacking.predict(Strak_X_test)

Predict Stacking-RF...


In [7]:
len(subA_Stacking)

2344

In [8]:
data_Y['FLAG'] = subA_Stacking
data_Y[['TICKER_SYMBOL','FLAG']].to_csv('stacking_制造业.csv')

In [9]:
len(subA_Stacking)

2344

In [10]:
len(subA_Stacking[subA_Stacking == 1])

237

In [11]:
len(subA_Stacking[subA_Stacking != 1])

2107

# 其他行业的stacking

In [17]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE   # SMOTE采样
from collections import Counter
from sklearn.ensemble import AdaBoostClassifier
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
from sklearn.ensemble import RandomForestClassifier  # 随机森林
import lightgbm as lgb # lightgbm
from sklearn.ensemble import GradientBoostingClassifier # GBDT
from sklearn.linear_model import LogisticRegression # 逻辑回归
from sklearn.tree import DecisionTreeClassifier 
from sklearn.svm import SVC
# import xgboost
# from xgboost import XGBClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score



os.getcwd()  # 'C:\\Users\\dell\\zhaoxuan'
os.listdir()  # 列出所有文件

filelist = os.listdir()

csv_list = [x for x in filelist if '数据.csv' in x]
# print(csv_list)
# print(csv_list[0])

for i in range(len(csv_list)):
    print(csv_list[i][:-11])
    col_list = pd.read_csv('../指标/300次训练.csv',encoding='gbk',header=None)
    new_col = list(col_list[col_list[0]==(csv_list[i][:-11])][1])  # 注意文件要求
    new_col.append('TICKER_SYMBOL')
    new_col.append('FLAG')
    print(new_col)
    data_rate = pd.read_csv(csv_list[i])
    data_rate = data_rate[new_col]
    data_X = data_rate[~data_rate['FLAG'].isnull()]
    data_Y = data_rate[data_rate['FLAG'].isnull()]
    data_X['FLAG'] = data_X['FLAG'].apply(lambda x :int(x))
    X_1 = data_X[data_X['FLAG']==1]
    X_0 = data_X[data_X['FLAG']==0].sample(X_1.shape[0]*3)
    X_0 = X_0.append(X_1)
    print(data_rate.shape)
    print(X_1.shape)
    if X_1.shape[0] <=1:
        print(csv_list[i],'停止运算')
        continue
    if data_rate.shape[1] <=3:
        continue

    X_sos, y_sos  = X_0.iloc[:,:-2],X_0.iloc[:,-1]

    #划分训练集和测试集
    x_train,x_val,y_train,y_val = train_test_split(X_0.iloc[:,:-2],X_0.iloc[:,-1],test_size=0.25) 

    #训练模型
    print('Predict GBDT...')
    model_gbdt = build_model_gbdt(x_train,y_train)
    val_gbdt = model_gbdt.predict(x_val)
    subA_gbdt = model_gbdt.predict(data_Y.iloc[:,:-2])


    print('predict XGB...')
    model_xgb = build_model_xgb(x_train,y_train)
    val_xgb = model_xgb.predict(x_val)
    subA_xgb = model_xgb.predict(data_Y.iloc[:,:-2])
    y_score_xgb = model_xgb.predict_proba(x_val)[:,1]
    fpr_xgb,tpr_xgb,thresholds_gbdt = roc_curve(y_val, y_score_xgb, pos_label=1)
    roc_auc_xgb = auc(fpr_xgb,tpr_xgb)  # 计算auc的值
    print(roc_auc_xgb)

    print('predict lgb...')
    model_lgb = build_model_lgb(x_train,y_train)
    val_lgb = model_lgb.predict(x_val)
    subA_lgb = model_lgb.predict(data_Y.iloc[:,:-2])

    #Starking
    #第一层
    train_lgb_pred = model_lgb.predict(x_train)
    train_xgb_pred = model_xgb.predict(x_train)
    train_gbdt_pred = model_gbdt.predict(x_train)

    Strak_X_train = pd.DataFrame()
    Strak_X_train['Method_1'] = train_lgb_pred
    Strak_X_train['Method_2'] = train_xgb_pred
    Strak_X_train['Method_3'] = train_gbdt_pred

    Strak_X_val = pd.DataFrame()
    Strak_X_val['Method_1'] = val_lgb
    Strak_X_val['Method_2'] = val_xgb
    Strak_X_val['Method_3'] = val_gbdt

    Strak_X_test = pd.DataFrame()
    Strak_X_test['Method_1'] = subA_lgb
    Strak_X_test['Method_2'] = subA_xgb
    Strak_X_test['Method_3'] = subA_gbdt

    #第二层
    model_lr_Stacking = build_model_RF(Strak_X_train,y_train)

    #训练集
    train_pre_Stacking = model_lr_Stacking.predict(Strak_X_train)
    # roc_auc_score(Strak_X_val, model_lr_Stacking.decision_function(Strak_X_train))
    # y_score_1 = model_lr_Stacking.predict_proba(Strak_X_val)[:,1]
    # fpr_1,tpr_1,thresholds_q = roc_curve(y_val, y_score_1, pos_label=1)
    # roc_auc_1 = auc(fpr_1,tpr_1)  # 计算auc的值
    # print('ROC-AUC:',roc_auc_1)


    #验证集
    val_pre_Stacking = model_lr_Stacking.predict(Strak_X_val)
    # print('ROC-AUC:',mean_absolute_error(y_val,val_pre_Stacking))

    #预测集
    print('Predict Stacking-RF...')
    subA_Stacking = model_lr_Stacking.predict(Strak_X_test)

#     clf_adaboost = AdaBoostClassifier()
#     clf_adaboost.fit(X_sos, y_sos)
#     pre_q2 = clf_adaboost.predict(data_Y.iloc[:,:-2])
    data_Y['FLAG'] = subA_Stacking
    data_Y.iloc[:,-2:].to_csv('{}stacking的问题3.csv'.format(i))

交通运输、仓储和邮政业
['T_LIAB', 'CAPITAL_RESER', 'T_NCL', 'PAID_IN_CAPITAL', 'SURPLUS_RESER', 'T_CL', 'DEFER_REVENUE', 'LT_BORR', 'RETAINED_EARNINGS', 'DEFER_TAX_LIAB', 'T_EQUITY_ATTR_P', 'LT_PAYABLE', 'T_LIAB_EQUITY', 'NCL_WITHIN_1Y', 'PREPAYMENT', 'MINORITY_INT', 'T_SH_EQUITY', 'AR', 'NOTES_RECEIV', 'OTH_CA', 'OTH_PAYABLE', 'C_PAID_OTH_FINAN_A', 'OTH_COMPRE_INCOME', 'OTH_RECEIV', 'T_CA', 'LT_EQUITY_INVEST', 'DIV_PAYABLE', 'INVENTORIES', 'FIXED_ASSETS', 'N_CF_FR_INVEST_A', 'TICKER_SYMBOL', 'FLAG']
(541, 32)
(3, 32)
Predict GBDT...
{'learning_rate': 0.05}
predict XGB...
[13:22:34] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
nan
predict lgb...
Predict Stacking-RF...
住宿和餐饮业
['TICKER_SYMBOL', 'FLAG']
(56, 2)
(0, 2)
住宿和餐饮业清洗完成的数据.csv 停止运算
信息传输、软件和信

Predict Stacking-RF...
水利、环境和公共设施管理业
['TAXES_PAYABLE', 'OTH_PAYABLE', 'NCL_WITHIN_1Y', 'T_CL', 'LT_BORR', 'DEFER_REVENUE', 'T_NCL', 'T_LIAB', 'PAID_IN_CAPITAL', 'CAPITAL_RESER', 'SURPLUS_RESER', 'RETAINED_EARNINGS', 'T_EQUITY_ATTR_P', 'INT_PAYABLE', 'NOTES_RECEIV', 'AR', 'ADVANCE_RECEIPTS', 'MINORITY_INT', 'PREPAYMENT', 'OTH_RECEIV', 'T_SH_EQUITY', 'C_FR_BORR', 'INVENTORIES', 'LT_EQUITY_INVEST', 'OTH_CA', 'C_FR_OTH_FINAN_A', 'T_COGS', 'PAYROLL_PAYABLE', 'T_LIAB_EQUITY', 'C_PAID_OTH_FINAN_A', 'TICKER_SYMBOL', 'FLAG']
(307, 32)
(2, 32)
Predict GBDT...
{'learning_rate': 0.05}
predict XGB...
[13:22:44] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
0.5
predict lgb...
Predict Stacking-RF...
电力、热力、燃气及水生产和供应业
['C_FR_OTH_INVEST_A', 'C_OUTF_OPERAT